In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from lenet import Lenet
from dataset import chunks
from datetime import datetime
import convolution as c
import numpy as np

iterations = 2000
batch_size = 50
display_step = 100 # summary save step
show_step = 10*display_step # print accuracy step
lr = 1e-4

mnist = input_data.read_data_sets('/tmp/dataset', one_hot=True)
test = list(chunks(mnist.test.images, 50)), list(chunks(mnist.test.labels, 50))
path = '/tmp/mnist/'
writer = tf.summary.FileWriter(path)

Extracting /tmp/dataset/train-images-idx3-ubyte.gz
Extracting /tmp/dataset/train-labels-idx1-ubyte.gz
Extracting /tmp/dataset/t10k-images-idx3-ubyte.gz
Extracting /tmp/dataset/t10k-labels-idx1-ubyte.gz


In [2]:
def train(sess, data, train_step, batch, prob, lr):
    train_step.run(session=sess, feed_dict={data[0]: batch[0], 
                                            data[1]: batch[1], 
                                            data[2]: prob, 
                                            data[3]: lr})
    
def show_acc(sess, data, accuracy, iteration):
    acc = []
    for i in range(len(test[0])):
        acc.append(accuracy.eval(session=sess, feed_dict={data[0]: test[0][i],
                                                          data[1]: test[1][i], 
                                                          data[2]: 1}))
    acc = np.mean(acc)
    if iteration % show_step == 0:
        print("step: {} accuracy: {:.4f}".format(iteration, acc))
    return acc
    
def save_summaries(sess, data, train_summary, test_summary, writer, batch, acc, iteration):
        s = train_summary.eval(session=sess, feed_dict={data[0]: batch[0],
                                                        data[1]: batch[1],
                                                        data[2]: 1})
        writer.add_summary(s, iteration)
        s = test_summary.eval(session=sess, feed_dict={data[4]: acc})
        writer.add_summary(s, iteration)
        writer.flush()
        
def run(scope, no, rate=1, conv=None):
    
    print("Running model: {} FM Rate: {}".format(scope, rate))
    
    x = tf.placeholder(tf.float32, [None, 784])
    keep_prob = tf.placeholder(tf.float32)
    learning_rate = tf.placeholder(tf.float32)
    labels = tf.placeholder(tf.float32, [None, 10])
    
    scalar = tf.placeholder(tf.float32) # for summary
    
    data = [x, labels, keep_prob, learning_rate, scalar]

    if conv is None:
        net = Lenet(x, keep_prob, device='/gpu:0', rate=rate)
    else:
        net = Lenet(x, keep_prob, device='/gpu:0', conv=conv, rate=rate)
        
    predict = net.y

    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict, labels=labels))
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(predict,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    

    with tf.name_scope(scope):
        with tf.name_scope(str(rate)):
            with tf.name_scope(str(no)):
                with tf.name_scope("test"):
                    test_summary = tf.summary.scalar('accuracy', scalar)

                with tf.name_scope("train"):
                    train1 = tf.summary.scalar('accuracy', accuracy)
                    train2 = tf.summary.scalar('loss', cross_entropy)
                    train_summary = tf.summary.merge([train1, train2])

    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)
    sess.as_default()
    sess.run(tf.global_variables_initializer())
    
    for iteration in range(iterations):
        batch = mnist.train.next_batch(batch_size)
        if iteration % display_step == 0:
            acc = show_acc(sess, data, accuracy, iteration)
            save_summaries(sess, data, train_summary, test_summary, writer, batch, acc, iteration)
        train(sess, data, train_step, batch, 0.5, lr)

    acc = show_acc(sess, data, accuracy, iterations)
    save_summaries(sess, data, train_summary, test_summary, writer, batch, acc, iterations)

    sess.close()


In [ ]:
run('basic', 0)

Running model: basic FM Rate: 1


In [3]:
run('conv_bin_scalar', 0, rate=1, conv=c.conv_bin_weights_scalar)

Running model: conv_bin_scalar FM Rate: 1


InvalidArgumentError: Cannot assign a device to node 'Variable_7/Adam_1': Could not satisfy explicit device specification '/device:GPU:0' because no devices matching that specification are registered in this process; available devices: /job:localhost/replica:0/task:0/cpu:0
Colocation Debug Info:
Colocation group had the following types and devices: 
Assign: CPU 
ApplyAdam: CPU 
Identity: CPU 
VariableV2: CPU 
	 [[Node: Variable_7/Adam_1 = VariableV2[_class=["loc:@Variable_7"], container="", dtype=DT_FLOAT, shape=[10], shared_name="", _device="/device:GPU:0"]()]]

Caused by op 'Variable_7/Adam_1', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py